In [1]:
%pip install transformers datasets
%pip install langdetect
%pip install jupyter
%pip install pandas
%pip install pyarrow
%pip install -U sentence-transformers
%pip install lexrank
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
# import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# table = pq.read_table("./data/mayo-disease-symptom.parquet")
# df = table.to_pandas()
# df = df.reindex(columns=["label", "text"])
# df.columns = ["disease", "conditions"]
# df
df = pd.read_csv("./data/merged_df.csv")
df = df.groupby(["CUI"]).agg({"STR": "first", "DEF": lambda x: " ".join(x)})
# df = df["DEF"].apply(lambda x: " ".join(x))
df.to_csv("data/CUI_STR_DEF.csv")

In [1]:
# from sentence_transformers import SentenceTransformer
# from sentence_transformers import util

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# sentence = df["STR"][1]
# sentences = df[2:]
# embeddings1 = model.encode(df["DEF"][1], convert_to_tensor=True)
# embeddings2 = model.encode(sentences["DEF"].to_numpy(), convert_to_tensor=True)

# cosine_scores = util.cos_sim(embeddings1, embeddings2)
# for i in range(len(sentences)):
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentence, sentences["STR"][i], cosine_scores[0][i]))

In [45]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
paraphrases = util.paraphrase_mining(model, df["DEF"], show_progress_bar=True, top_k=7)
# embeddings = model.encode(df["DEF"][0:100], convert_to_tensor=True)

result = []
for paraphrase in paraphrases:
    score, i, j = paraphrase
    result.append([df.index[i],
            df["STR"][i],
            df.index[j],
            df["STR"][j],
            score])
    # print("{} \t\t {} \t\t Score: {:.4f}".format(df["STR"][i], df["STR"][j], score))
result_df = pd.DataFrame(result, columns=["CUI_1", "STR_1", "CUI_2", "STR_2", "SCORE"])
result_df = result_df.sort_values(["CUI_1", "SCORE"], ascending=[True, False])
result_df.to_csv("output/similarity.csv", index=False)

Batches: 100%|██████████| 45/45 [00:56<00:00,  1.25s/it]


In [4]:
import pandas as pd
similarity_df = pd.read_csv("output/similarity.csv")
similarity_df


def get_similar_diseases(similarity_df, name: str):
    return similarity_df.loc[similarity_df["STR_1"] == name]

get_similar_diseases(similarity_df, "ABETALIPOPROTEINEMIA")

,CUI_1,STR_1,CUI_2,STR_2,SCORE
0,C0000744,ABETALIPOPROTEINEMIA,C0020479,HYPERLIPOPROTEINEMIA TYPE III,0.734213
1,C0000744,ABETALIPOPROTEINEMIA,C0023817,HYPERLIPOPROTEINEMIA TYPE I,0.732668
2,C0000744,ABETALIPOPROTEINEMIA,C0020480,HYPERLIPOPROTEINEMIA TYPE IV,0.729844
3,C0000744,ABETALIPOPROTEINEMIA,C0028064,NIEMANN-PICK DISEASE,0.670821
4,C0000744,ABETALIPOPROTEINEMIA,C0020445,HYPERLIPOPROTEINEMIA TYPE II,0.650171
5,C0000744,ABETALIPOPROTEINEMIA,C0034960,REFSUM DISEASE,0.640331
6,C0000744,ABETALIPOPROTEINEMIA,C0023195,LECITHIN CHOLESTEROL ACYLTRANSFERASE DEFICIENCY,0.638767
